In [42]:
import pandas as pd

df = pd.read_csv('./datasets/wine.csv', header=None)
df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,1,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065
1,1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050
2,1,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185
3,1,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480
4,1,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
173,3,13.71,5.65,2.45,20.5,95,1.68,0.61,0.52,1.06,7.70,0.64,1.74,740
174,3,13.40,3.91,2.48,23.0,102,1.80,0.75,0.43,1.41,7.30,0.70,1.56,750
175,3,13.27,4.28,2.26,20.0,120,1.59,0.69,0.43,1.35,10.20,0.59,1.56,835
176,3,13.17,2.59,2.37,20.0,120,1.65,0.68,0.53,1.46,9.30,0.60,1.62,840


In [43]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 178 entries, 0 to 177
Data columns (total 14 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       178 non-null    int64  
 1   1       178 non-null    float64
 2   2       178 non-null    float64
 3   3       178 non-null    float64
 4   4       178 non-null    float64
 5   5       178 non-null    int64  
 6   6       178 non-null    float64
 7   7       178 non-null    float64
 8   8       178 non-null    float64
 9   9       178 non-null    float64
 10  10      178 non-null    float64
 11  11      178 non-null    float64
 12  12      178 non-null    float64
 13  13      178 non-null    int64  
dtypes: float64(11), int64(3)
memory usage: 19.6 KB


In [44]:
df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,1,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065
1,1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050
2,1,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185
3,1,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480
4,1,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
173,3,13.71,5.65,2.45,20.5,95,1.68,0.61,0.52,1.06,7.70,0.64,1.74,740
174,3,13.40,3.91,2.48,23.0,102,1.80,0.75,0.43,1.41,7.30,0.70,1.56,750
175,3,13.27,4.28,2.26,20.0,120,1.59,0.69,0.43,1.35,10.20,0.59,1.56,835
176,3,13.17,2.59,2.37,20.0,120,1.65,0.68,0.53,1.46,9.30,0.60,1.62,840


## ModelCheckpoint, EarlyStopping

In [45]:
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.utils import to_categorical

import numpy
import os
import tensorflow as tf

# seed 값 설정
numpy.random.seed(3)
tf.random.set_seed(3)

x = df.drop(columns=0, axis=1)
y = df[0]
x_scaled = StandardScaler().fit_transform(x)

model = Sequential()
model.add(Dense(30, input_dim=13, activation='relu'))
model.add(Dense(12, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
             optimizer='adam',
             metrics=['accuracy'])

# 모델 저장 폴더 만들기
MODEL_DIR = './model/'
if not os.path.exists(MODEL_DIR):
    os.mkdir(MODEL_DIR)
modelpath="./model/{epoch:02d}-{val_loss:4f}.hdf5"

# 모델 업데이트 및 저장
checkpointer = ModelCheckpoint(filepath=modelpath, monitor='val_loss', verbose=1, save_best_only=True)

# 학습 자동 중단 설정
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=100)

model.fit(x_scaled, y, validation_split=0.2, epochs=3500, batch_size=10,
         verbose=1, callbacks=[early_stopping_callback, checkpointer])


Epoch 1/3500
1/1 [==============================] - ETA: 0s - loss: 0.6521 - accuracy: 0.3803
Epoch 1: val_loss improved from inf to 0.80518, saving model to ./model\01-0.805181.hdf5
1/1 [==============================] - 1s 814ms/step - loss: 0.6521 - accuracy: 0.3803 - val_loss: 0.8052 - val_accuracy: 0.0000e+00
Epoch 2/3500
1/1 [==============================] - ETA: 0s - loss: 0.6325 - accuracy: 0.3873
Epoch 2: val_loss improved from 0.80518 to 0.77398, saving model to ./model\02-0.773977.hdf5
1/1 [==============================] - 0s 47ms/step - loss: 0.6325 - accuracy: 0.3873 - val_loss: 0.7740 - val_accuracy: 0.0000e+00
Epoch 3/3500
1/1 [==============================] - ETA: 0s - loss: 0.6136 - accuracy: 0.3873
Epoch 3: val_loss improved from 0.77398 to 0.74182, saving model to ./model\03-0.741825.hdf5
1/1 [==============================] - 0s 52ms/step - loss: 0.6136 - accuracy: 0.3873 - val_loss: 0.7418 - val_accuracy: 0.0000e+00
Epoch 4/3500
1/1 [============================

1/1 [==============================] - 0s 61ms/step - loss: 0.1114 - accuracy: 0.4155 - val_loss: -0.0353 - val_accuracy: 0.0000e+00
Epoch 27/3500
1/1 [==============================] - ETA: 0s - loss: 0.0840 - accuracy: 0.4155
Epoch 27: val_loss improved from -0.03533 to -0.08332, saving model to ./model\27--0.083322.hdf5
1/1 [==============================] - 0s 55ms/step - loss: 0.0840 - accuracy: 0.4155 - val_loss: -0.0833 - val_accuracy: 0.0000e+00
Epoch 28/3500
1/1 [==============================] - ETA: 0s - loss: 0.0553 - accuracy: 0.4155
Epoch 28: val_loss improved from -0.08332 to -0.13407, saving model to ./model\28--0.134073.hdf5
1/1 [==============================] - 0s 47ms/step - loss: 0.0553 - accuracy: 0.4155 - val_loss: -0.1341 - val_accuracy: 0.0000e+00
Epoch 29/3500
1/1 [==============================] - ETA: 0s - loss: 0.0257 - accuracy: 0.4155
Epoch 29: val_loss improved from -0.13407 to -0.18763, saving model to ./model\29--0.187633.hdf5
1/1 [====================

Epoch 52/3500
1/1 [==============================] - ETA: 0s - loss: -0.9977 - accuracy: 0.4155
Epoch 52: val_loss improved from -2.20286 to -2.33485, saving model to ./model\52--2.334852.hdf5
1/1 [==============================] - 0s 52ms/step - loss: -0.9977 - accuracy: 0.4155 - val_loss: -2.3349 - val_accuracy: 0.0000e+00
Epoch 53/3500
1/1 [==============================] - ETA: 0s - loss: -1.0566 - accuracy: 0.4155
Epoch 53: val_loss improved from -2.33485 to -2.46853, saving model to ./model\53--2.468527.hdf5
1/1 [==============================] - 0s 47ms/step - loss: -1.0566 - accuracy: 0.4155 - val_loss: -2.4685 - val_accuracy: 0.0000e+00
Epoch 54/3500
1/1 [==============================] - ETA: 0s - loss: -1.1168 - accuracy: 0.4155
Epoch 54: val_loss improved from -2.46853 to -2.60418, saving model to ./model\54--2.604183.hdf5
1/1 [==============================] - 0s 54ms/step - loss: -1.1168 - accuracy: 0.4155 - val_loss: -2.6042 - val_accuracy: 0.0000e+00
Epoch 55/3500
1/1 [

1/1 [==============================] - ETA: 0s - loss: -2.9772 - accuracy: 0.4155
Epoch 77: val_loss improved from -6.30666 to -6.51444, saving model to ./model\77--6.514443.hdf5
1/1 [==============================] - 0s 50ms/step - loss: -2.9772 - accuracy: 0.4155 - val_loss: -6.5144 - val_accuracy: 0.0000e+00
Epoch 78/3500
1/1 [==============================] - ETA: 0s - loss: -3.0873 - accuracy: 0.4155
Epoch 78: val_loss improved from -6.51444 to -6.72739, saving model to ./model\78--6.727394.hdf5
1/1 [==============================] - 0s 44ms/step - loss: -3.0873 - accuracy: 0.4155 - val_loss: -6.7274 - val_accuracy: 0.0000e+00
Epoch 79/3500
1/1 [==============================] - ETA: 0s - loss: -3.2006 - accuracy: 0.4155
Epoch 79: val_loss improved from -6.72739 to -6.94529, saving model to ./model\79--6.945288.hdf5
1/1 [==============================] - 0s 51ms/step - loss: -3.2006 - accuracy: 0.4155 - val_loss: -6.9453 - val_accuracy: 0.0000e+00
Epoch 80/3500
1/1 [==============

Epoch 102/3500
1/1 [==============================] - ETA: 0s - loss: -7.2293 - accuracy: 0.4155
Epoch 102: val_loss improved from -13.66580 to -14.09141, saving model to ./model\102--14.091413.hdf5
1/1 [==============================] - 0s 52ms/step - loss: -7.2293 - accuracy: 0.4155 - val_loss: -14.0914 - val_accuracy: 0.0000e+00
Epoch 103/3500
1/1 [==============================] - ETA: 0s - loss: -7.4865 - accuracy: 0.4155
Epoch 103: val_loss improved from -14.09141 to -14.53640, saving model to ./model\103--14.536404.hdf5
1/1 [==============================] - 0s 47ms/step - loss: -7.4865 - accuracy: 0.4155 - val_loss: -14.5364 - val_accuracy: 0.0000e+00
Epoch 104/3500
1/1 [==============================] - ETA: 0s - loss: -7.7522 - accuracy: 0.4155
Epoch 104: val_loss improved from -14.53640 to -15.00047, saving model to ./model\104--15.000468.hdf5
1/1 [==============================] - 0s 43ms/step - loss: -7.7522 - accuracy: 0.4155 - val_loss: -15.0005 - val_accuracy: 0.0000e+0

...

1/1 [==============================] - ETA: 0s - loss: -7211964.5000 - accuracy: 0.4155
Epoch 3393: val_loss improved from -10052386.00000 to -10061911.00000, saving model to ./model\3393--10061911.000000.hdf5
1/1 [==============================] - 0s 89ms/step - loss: -7211964.5000 - accuracy: 0.4155 - val_loss: -10061911.0000 - val_accuracy: 0.0000e+00
Epoch 3394/3500
1/1 [==============================] - ETA: 0s - loss: -7218879.0000 - accuracy: 0.4155
Epoch 3394: val_loss improved from -10061911.00000 to -10071445.00000, saving model to ./model\3394--10071445.000000.hdf5
1/1 [==============================] - 0s 53ms/step - loss: -7218879.0000 - accuracy: 0.4155 - val_loss: -10071445.0000 - val_accuracy: 0.0000e+00
Epoch 3395/3500
1/1 [==============================] - ETA: 0s - loss: -7225798.5000 - accuracy: 0.4155
Epoch 3395: val_loss improved from -10071445.00000 to -10080983.00000, saving model to ./model\3395--10080983.000000.hdf5
1/1 [==============================] - 0s 52

1/1 [==============================] - ETA: 0s - loss: -7365113.0000 - accuracy: 0.4155
Epoch 3415: val_loss improved from -10263214.00000 to -10272839.00000, saving model to ./model\3415--10272839.000000.hdf5
1/1 [==============================] - 0s 53ms/step - loss: -7365113.0000 - accuracy: 0.4155 - val_loss: -10272839.0000 - val_accuracy: 0.0000e+00
Epoch 3416/3500
1/1 [==============================] - ETA: 0s - loss: -7372126.5000 - accuracy: 0.4155
Epoch 3416: val_loss improved from -10272839.00000 to -10282467.00000, saving model to ./model\3416--10282467.000000.hdf5
1/1 [==============================] - 0s 57ms/step - loss: -7372126.5000 - accuracy: 0.4155 - val_loss: -10282467.0000 - val_accuracy: 0.0000e+00
Epoch 3417/3500
1/1 [==============================] - ETA: 0s - loss: -7379143.0000 - accuracy: 0.4155
Epoch 3417: val_loss improved from -10282467.00000 to -10292099.00000, saving model to ./model\3417--10292099.000000.hdf5
1/1 [==============================] - 0s 56

1/1 [==============================] - ETA: 0s - loss: -7520413.0000 - accuracy: 0.4155
Epoch 3437: val_loss improved from -10476169.00000 to -10485911.00000, saving model to ./model\3437--10485911.000000.hdf5
1/1 [==============================] - 0s 45ms/step - loss: -7520413.0000 - accuracy: 0.4155 - val_loss: -10485911.0000 - val_accuracy: 0.0000e+00
Epoch 3438/3500
1/1 [==============================] - ETA: 0s - loss: -7527523.0000 - accuracy: 0.4155
Epoch 3438: val_loss improved from -10485911.00000 to -10495655.00000, saving model to ./model\3438--10495655.000000.hdf5
1/1 [==============================] - 0s 55ms/step - loss: -7527523.0000 - accuracy: 0.4155 - val_loss: -10495655.0000 - val_accuracy: 0.0000e+00
Epoch 3439/3500
1/1 [==============================] - ETA: 0s - loss: -7534639.5000 - accuracy: 0.4155
Epoch 3439: val_loss improved from -10495655.00000 to -10505404.00000, saving model to ./model\3439--10505404.000000.hdf5
1/1 [==============================] - 0s 16

1/1 [==============================] - ETA: 0s - loss: -7677872.5000 - accuracy: 0.4155
Epoch 3459: val_loss improved from -10691622.00000 to -10701486.00000, saving model to ./model\3459--10701486.000000.hdf5
1/1 [==============================] - 0s 53ms/step - loss: -7677872.5000 - accuracy: 0.4155 - val_loss: -10701486.0000 - val_accuracy: 0.0000e+00
Epoch 3460/3500
1/1 [==============================] - ETA: 0s - loss: -7685079.5000 - accuracy: 0.4155
Epoch 3460: val_loss improved from -10701486.00000 to -10711355.00000, saving model to ./model\3460--10711355.000000.hdf5
1/1 [==============================] - 0s 50ms/step - loss: -7685079.5000 - accuracy: 0.4155 - val_loss: -10711355.0000 - val_accuracy: 0.0000e+00
Epoch 3461/3500
1/1 [==============================] - ETA: 0s - loss: -7692293.5000 - accuracy: 0.4155
Epoch 3461: val_loss improved from -10711355.00000 to -10721232.00000, saving model to ./model\3461--10721232.000000.hdf5
1/1 [==============================] - 0s 53

1/1 [==============================] - ETA: 0s - loss: -7837488.0000 - accuracy: 0.4155
Epoch 3481: val_loss improved from -10910158.00000 to -10920169.00000, saving model to ./model\3481--10920169.000000.hdf5
1/1 [==============================] - 0s 52ms/step - loss: -7837488.0000 - accuracy: 0.4155 - val_loss: -10920169.0000 - val_accuracy: 0.0000e+00
Epoch 3482/3500
1/1 [==============================] - ETA: 0s - loss: -7844794.5000 - accuracy: 0.4155
Epoch 3482: val_loss improved from -10920169.00000 to -10930186.00000, saving model to ./model\3482--10930186.000000.hdf5
1/1 [==============================] - 0s 62ms/step - loss: -7844794.5000 - accuracy: 0.4155 - val_loss: -10930186.0000 - val_accuracy: 0.0000e+00
Epoch 3483/3500
1/1 [==============================] - ETA: 0s - loss: -7852106.0000 - accuracy: 0.4155
Epoch 3483: val_loss improved from -10930186.00000 to -10940209.00000, saving model to ./model\3483--10940209.000000.hdf5
1/1 [==============================] - 0s 56